In [1]:
%cd ..
%pwd

/Users/amiara/NAWI/WaterTap/watertap/NAWI-WaterTAP3/watertap3/watertap3


'/Users/amiara/NAWI/WaterTap/watertap/NAWI-WaterTAP3/watertap3/watertap3'

In [2]:
from watertap3.utils import watertap_setup, get_case_study, run_water_tap, run_water_tap_ro, show_train2, get_fixed_onm_reduction, run_sensitivity, get_results_table
import pandas as pd

In [3]:
df = pd.read_csv("data/baseline_cases_runs.csv")
reference = "nawi"

In [4]:
for source_water in ["brackish"]: #df.source_water_category.unique():
    for case_study in df[df.source_water_category == source_water].case_study.unique():

        h1 = df[df.case_study == case_study]
        scenarios = h1.scenario.unique()
        for scenario in scenarios: 

            print("------------------------------------------------------")
            print("------------------------------------------------------")
            print("-----------", case_study, scenario, "-----------------")
            print("------------------------------------------------------")
            print("------------------------------------------------------")

            desired_recovery = df[((df.case_study == case_study) & (df.scenario == scenario))].max_recovery_rate.max()

            source_water_category = df[((df.case_study == case_study) & (df.scenario == scenario))].source_water_category.max()

            skip_small_base = df[((df.case_study == case_study) & (df.scenario == scenario))].skip_small_base.max()
            skip_small_sens = df[((df.case_study == case_study) & (df.scenario == scenario))].skip_small_sens.max()
            ro_bounds = df[((df.case_study == case_study) & (df.scenario == scenario))].ro_bounds.max()
#             ro_bounds = 'seawater'

            m = watertap_setup(dynamic=False, case_study = case_study, reference = reference, 
                                  scenario = scenario, source_scenario = "baseline")

            m = get_case_study(m=m)

            if "1p5" in scenarios:
                m.fs.irrigation_and_drainage.flow_vol_in.fix(0.0657)
            
            ############################################################
            ############################################################
            # set recovery for uranium case RO units.
            if case_study == "uranium":
                m.fs.ion_exchange.removal_fraction[0, "tds"].unfix()
                m.fs.ion_exchange.water_recovery.fix(0.967)
                m.fs.ion_exchange.anion_res_capacity.unfix()
                m.fs.ion_exchange.cation_res_capacity.unfix()
            
            if scenario == "swd_well":
                m.fs.produced_water_for_dw.conc_mass_in[0, "tds"].fix(50)
            
            # run and and return baseline with 0 degrees of freedom
            m = run_water_tap_ro(m, source_water_category = source_water_category, 
                                 desired_recovery = desired_recovery, skip_small = skip_small_base, 
                                 ro_bounds = ro_bounds, source_scenario = "baseline", scenario_name = scenario)

            if scenario == "baseline":
                run_sensitivity(m = m, save_results = True, scenario = scenario, 
                                    case_study = case_study, skip_small_sens = skip_small_sens)

                if source_water == "seawater":
                    m = get_fixed_onm_reduction(m, reduction_value_list = [0.9, 0.8, 0.7, 0.6, 0.5],
                                                  skip_small=skip_small_base)

                if case_study == "san_luis":
                    n = 1
                    for flow_scenario in [0.0657, 0.1314, 0.2191]:
                        m.fs.irrigation_and_drainage.flow_vol_in.fix(flow_scenario)
                        for key in m.fs.pfd_dict.keys():
                            if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
                                getattr(m.fs, key).feed.pressure.unfix()
                                getattr(m.fs, key).membrane_area.unfix()

                        print("adjusted baseline flow -->", flow_scenario)
                        run_water_tap(m=m, objective=True, skip_small=skip_small_base, print_model_results="summary")
                        if n == 1: s_name = "1p5_mgd"
                        if n == 2: s_name = "3_mgd"
                        if n == 3: s_name = "5_mgd"
                        df_no2 = get_results_table(m=m, case_study=m.fs.train["case_study"], scenario=s_name)                
                        n = n + 1
                        
            
            if case_study == "lithium":
                ### double cap
                if scenario == "baseline": 
                    s_name = "double_cap"
                else:
                    s_name = scenario + "_double_cap"
                print("goes in")
                flow_in_x2 = m.fs.lithium_brine.flow_vol_in[0]() * 2
                m.fs.lithium_brine.flow_vol_in.fix(flow_in_x2)
                print("adjusted baseline flow -->", flow_in_x2)
                run_water_tap(m=m, objective=True, skip_small=skip_small_base, print_model_results="summary")
                df_no3 = get_results_table(m=m, case_study=m.fs.train["case_study"], scenario=s_name)

            
            if case_study == 'produced_water_injection' and scenario == 'swd_well':
                m.fs.brine_concentrator.water_recovery.fix(0.725)
            
            dwi_list = ["emwd", "big_spring"]   
            if m.fs.train["case_study"] in dwi_list:
                for key in m.fs.pfd_dict.keys():
                    if m.fs.pfd_dict[key]["Unit"] == "deep_well_injection":
                        run_sensitivity(m = m, save_results = True, scenario = scenario, 
                                            case_study = case_study, skip_small_sens = skip_small_sens)

            if scenario in ["edr_ph_ro", "ro_and_mf"]:
                run_sensitivity(m = m, save_results = True, scenario = scenario, 
                                    case_study = case_study, skip_small_sens = skip_small_sens)                

            if source_water == "power":
                for key in m.fs.pfd_dict.keys():
                    if m.fs.pfd_dict[key]["Unit"] == "evaporation_pond":
                        run_sensitivity(m = m, save_results = True, scenario = scenario, 
                                            case_study = case_study, skip_small_sens = skip_small_sens)  



------------------------------------------------------
------------------------------------------------------
----------- emwd baseline -----------------
------------------------------------------------------
------------------------------------------------------
EMWD

------- Adding Unit Processes -------
WELL FIELD
IRON AND MANGANESE REMOVAL
CARTRIDGE FILTRATION A
RO A1
RO A2
DECARBONATOR A
CARTRIDGE FILTRATION B
RO B1
RO B2
DECARBONATOR B
CHLORINATION
TREATED STORAGE
MUNICIPAL DRINKING
SURFACE DISCHARGE
-------------------------------------

params into splitter --> [0.033, 0.467, 0.116, 0.384]
params into splitter --> [0.5, 0.5]
params into splitter --> [0.5, 0.5]

Connecting unit processes...

Unfixing feed pressure and area for RO A1...


Unfixing feed pressure and area for RO A2...


Unfixing feed pressure and area for RO B1...


Unfixing feed pressure and area for RO B2...

Case Study = EMWD
Scenario = BASELINE

------------------------------------------------------------------

------- Plant Capacity Utilization 70-100% -------
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.7 LCOW --> 0.8336503179708816
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.715 LCOW --> 0.8161202024002301
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.73 LCOW --> 0.7993497721332788
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

------------------------------------


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Weighted Average Cost of Capital 5-10% 0.07000000000000002 LCOW --> 0.6447963734924247
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Weighted Average Cost of Capital 5-10% 0.07250000000000002 LCOW --> 0.652815534254021
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Weighted Average Cost of Capital 5-10% 0.07500000000000002 LCOW --> 0.6609072482935534
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Weighted Average Cost of Capital 5-10% 0.0775000000000


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-25% 2.7377500000000006 LCOW --> 0.5977206664587429
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-25% 2.796000000000001 LCOW --> 0.5997456844988117
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-25% 2.8542500000000013 LCOW --> 0.6017707366686931
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-25% 2.912500000000001 LCOW --> 0.6037960524926339

------- RESET -------

----------------------------------------


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 22.5 LCOW --> 0.622886597008194
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 24.0 LCOW --> 0.6127882417802305
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 25.5 LCOW --> 0.60400151074664
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 27.0 LCOW --> 0.5963058012783212
----------------------------------------------------------------------

Degrees of Fr

...


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-20% 1.5367197000000006 LCOW --> 0.3102315694104213
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-20% 1.5701266500000004 LCOW --> 0.3124356399802642
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-20% 1.6035336000000007 LCOW --> 0.3146908003770992
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-20% 1.6369405500000007 LCOW --> 0.31698022985483815
---------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.04468999999999999 LCOW --> 0.2903428705610611
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.046324999999999984 LCOW --> 0.2918575238978441
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.04795999999999998 LCOW --> 0.29337217723462705
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.04959499999999998 LCOW --> 0.294886830571410

KBHDP

------- Adding Unit Processes -------
WELL FIELD
MEDIA FILTRATION
ANTI SCALANT ADDITION
RO FIRST STAGE
PASSTHROUGH
LIME SOFTENING
RO SECOND STAGE
CAUSTIC SODA ADDITION
CHLORINATION
TREATED STORAGE
EVAPORATION POND
LANDFILL ZLD
MUNICIPAL DRINKING
-------------------------------------

params into splitter --> [0.393, 0.607]

Connecting unit processes...
Unfixing feed pressure and area for RO FIRST STAGE ...

Unfixing feed pressure and area for RO SECOND STAGE ...

----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


WELL FIELD:

	total cap investment: 12.10508
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.1606

MEDIA FILTRATION:

	total cap investment: 7.92421
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.10552

ANTI SCALANT ADDITION:

	total cap invest

KBHDP

------- Adding Unit Processes -------
WELL FIELD
MEDIA FILTRATION
ANTI SCALANT ADDITION
RO FIRST STAGE
PASSTHROUGH
LIME SOFTENING
RO SECOND STAGE
CAUSTIC SODA ADDITION
CHLORINATION
TREATED STORAGE
BRINE CONCENTRATOR
EVAPORATION POND
LANDFILL ZLD
MUNICIPAL DRINKING
-------------------------------------

params into splitter --> [0.393, 0.607]

Connecting unit processes...
Unfixing feed pressure and area for RO FIRST STAGE ...

Unfixing feed pressure and area for RO SECOND STAGE ...

----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


WELL FIELD:

	total cap investment: 12.10508
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.1606

MEDIA FILTRATION:

	total cap investment: 7.92421
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.10552

ANTI SCALANT ADDITION:


WaterTAP3 solution optimal 

----------------------------------------------------------------------

	Feed pressure for RO FIRST STAGE: 25.0 bar
	Membrane area for RO FIRST STAGE: 91094.89 m2
	Pure Water Flux for RO FIRST STAGE: 25.42 LMH
	A constant for RO FIRST STAGE: 4.2
	B constant for RO FIRST STAGE: 0.35

	Recovery for RO FIRST STAGE: 0.8


	Feed pressure for RO SECOND STAGE: 25.0 bar
	Membrane area for RO SECOND STAGE: 46512.65 m2
	Pure Water Flux for RO SECOND STAGE: 8.0 LMH
	A constant for RO SECOND STAGE: 4.2
	B constant for RO SECOND STAGE: 0.35

	Recovery for RO SECOND STAGE: 0.8


Rebuilding model, running with unfixed, then fixed RO based on model results...

KBHDP

------- Adding Unit Processes -------
WELL FIELD
MEDIA FILTRATION
ANTI SCALANT ADDITION
RO FIRST STAGE
PASSTHROUGH
LIME SOFTENING
RO SECOND STAGE
CAUSTIC SODA ADDITION
CHLORINATION
TREATED STORAGE
BRINE CONCENTRATOR
CRYSTALLIZER
LANDFILL ZLD
MUNICIPAL DRINKING
-------------------------------------

params int

In [5]:
import pyomo.util.infeasible as infeas
print(infeas.log_infeasible_bounds(m))
print(infeas.log_infeasible_constraints(m))

None
INFO: CONSTR fs.brine_concentrator.recovery_equation[0.0]:
    -0.007238449530761408 =/= 0.0
None


In [ ]:
# case_study = 'ashkelon'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = 'seawater'
# ro_bounds = 'seawater'
# desired_recovery = 1

# case_study = 'carlsbad'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = 'seawater'
# ro_bounds = 'seawater'
# desired_recovery = 1

# case_study = 'santa_barbara'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = 'seawater'
# ro_bounds = 'seawater'
# desired_recovery = 1

# case_study = 'tampa_bay'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = 'seawater'
# ro_bounds = 'seawater'
# desired_recovery = 1

case_study = 'emwd'
scenario = 'baseline'
reference = 'nawi'
source_water = 'emwd_ca_brackish'
ro_bounds = 'other'
desired_recovery = 1

# case_study = 'irwin'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = 'brackish'
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'kbhdp'
# scenario = 'zld_c'
# reference = 'nawi'
# source_water = 'kbhdp_brackish_ave'
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'big_spring'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = 'big_spring_feed'
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'hrsd'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = 'hrsd_municipal'
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'ocwd'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = 'ocwd_feed'
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'solaire'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = ['solaire_graywater', 'solaire_blackwater']
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'heap_leaching'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = ['ore_heap_make_up', 'ore_heap_recycle']
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'lithium'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = 'lithium_brine'
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'uranium'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = ['uranium_active', 'uranium_restore']
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'cherokee'
# scenario = 'zld_brine_conc'
# reference = 'nawi'
# source_water = ['cherokee_ct_makeup', 'denver_city_water']
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'gila_river'
# scenario = 'both_ro'
# reference = 'nawi'
# source_water = 'gila_river_water'
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'beef'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = 'beef_processing'
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'monterey_one'
# scenario = 'add_ix'
# reference = 'nawi'
# source_water = 'secondary_ww'
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'san_luis'
# scenario = 'stage_1p5mgd_edr'
# reference = 'nawi'
# source_water = 'irrigation_and_drainage'
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'iron_and_steel'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = ['steel_oily_ww', 'steel_metal_ww', 'hot_mill_blowdown', 'hot_mill_blowdown_b']
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'pulp_and_paper'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = ['main_sewer', 'corrosive_sewer']
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'upw'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = ['fab25', 'upw_brine']
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'kern_county'
# scenario = 'ro_and_mf'
# reference = 'nawi'
# source_water = ['produced_water', 'surface_water']
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'damodar'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = 'powder_river_basin_cbm'
# ro_bounds = 'other'
# desired_recovery = 1

m = watertap_setup(dynamic=False, case_study=case_study, reference=reference, 
                      scenario=scenario, source_scenario="baseline")

m = get_case_study(m=m)

m = run_water_tap_ro(m, desired_recovery=desired_recovery, ro_bounds=ro_bounds,
                 source_water_category='other', scenario_name=scenario,
                 source_scenario='baseline')    

# run_water_tap(m, print_model_results='summary')